# Module 4: CASE Statement Examples (With Python Execution)

This section demonstrates practical use of SQL CASE statements with Python execution using `mysql.connector` and `pandas`.

In [1]:
import mysql.connector
import pandas as pd

# Establish connection to local MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",  # never use root in production
    password="William2025!!",  # replace with your secure password
    database="farmers_market"
)

cursor = conn.cursor()
print(f"Successfully Connected to {conn.database}!")

# check what I have added above in the message, it's better to user the database name! 
# Better be safe than sorry!!

Successfully Connected to farmers_market!


## 1. Grouping Continuous Values Using CASE

Classify purchase quantity into 'small', 'medium', or 'large'.

In [7]:
query = '''
SELECT
    market_date,
    transaction_time,
    quantity,
    CASE
        WHEN quantity <= 1
        THEN 'small'
        WHEN quantity > 1 AND quantity <= 3 
        THEN 'medium'
        WHEN quantity > 3 
        THEN 'large'
        ELSE NULL
    END AS purchase_category
FROM farmers_market.customer_purchases;
'''

cursor = conn.cursor()
cursor.execute(query)
df_quantity_grouped = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
cursor.close()

df_quantity_grouped

,market_date,transaction_time,quantity,purchase_category
0,2019-07-03,0 days 17:32:00,0.99,small
1,2019-07-03,0 days 18:23:00,2.18,medium
2,2019-07-03,0 days 18:41:00,1.53,medium
3,2019-07-03,0 days 18:18:00,2.02,medium
4,2019-07-03,0 days 17:34:00,0.66,small
...,...,...,...,...
4216,2020-09-30,0 days 18:48:00,2.00,medium
4217,2020-09-30,0 days 16:30:00,1.00,small
4218,2020-09-30,0 days 18:25:00,5.00,large
4219,2020-09-30,0 days 18:28:00,6.00,large


## 2. Creating Binary Flags Using CASE

Identify market days that experienced rain or snow.

In [10]:
query = '''
SELECT
    market_date,
    market_min_temp,
    market_max_temp,
    market_rain_flag,
    market_snow_flag,
    CASE
        WHEN market_rain_flag = 1 OR market_snow_flag = 1 
        THEN 'Yes'
        WHEN market_rain_flag = 0 AND market_snow_flag = 0 
        THEN 'No'
        ELSE NULL
    END AS weather_warning
FROM farmers_market.market_date_info;
'''

cursor = conn.cursor()
cursor.execute(query)
df_weather_warning = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
cursor.close()

df_weather_warning.head()

,market_date,market_min_temp,market_max_temp,market_rain_flag,market_snow_flag,weather_warning
0,2019-03-02,34,42,0,0,No
1,2019-03-09,28,32,0,1,Yes
2,2019-03-13,52,60,0,0,No
3,2019-03-16,33,45,0,0,No
4,2019-03-20,43,51,1,0,Yes


## 3. Categorical Ordinal Encoding Using CASE

Convert booth size category into ranked numeric values.

In [13]:
query = '''
SELECT
    booth_number,
    booth_type,
    CASE
        WHEN booth_type = 'Small'
        THEN 1
        WHEN booth_type = 'Standard' 
        THEN 2
        WHEN booth_type = 'Large' 
        THEN 3
        ELSE NULL
    END AS booth_type_encoded
FROM farmers_market.booth
-- lets also sort the results by booth_type_encoded
ORDER BY booth_type_encoded;
'''

cursor = conn.cursor()
cursor.execute(query)
df_booth_encoded = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
cursor.close()

df_booth_encoded

,booth_number,booth_type,booth_type_encoded
0,3,Small,1
1,4,Small,1
2,5,Small,1
3,6,Small,1
4,8,Small,1
5,9,Small,1
6,1,Standard,2
7,2,Standard,2
8,7,Standard,2
9,10,Large,3


## 4. Categorical One-Hot Encoding Using CASE

Turn non-ranked category `product_qty_type` into dummy variables.

In [15]:
query = '''
SELECT
    product_name,
    product_qty_type,
    CASE
        WHEN product_qty_type = 'lbs' 
        THEN 1
        ELSE 0
    END AS product_qty_type_lbs,
    CASE
        WHEN product_qty_type = 'unit' 
        THEN 1
        ELSE 0
    END AS product_qty_type_unit
FROM farmers_market.product
LIMIT 6;
'''

cursor = conn.cursor()
cursor.execute(query)
df_product_ohe = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
cursor.close()

df_product_ohe.head()

,product_name,product_qty_type,product_qty_type_lbs,product_qty_type_unit
0,Habanero Peppers - Organic,lbs,1,0
1,Jalapeno Peppers - Organic,lbs,1,0
2,Poblano Peppers - Organic,unit,0,1
3,Banana Peppers - Jar,unit,0,1
4,Whole Wheat Bread,unit,0,1
